1) y - categorial [1...50]

2) X_i_0 - categorial [0, 1, 2, 3] ~ 3

3) X_i_1 - categorial [-1, 0, 1, ... , 100] ~ 100

4) X_i_2 - [-1, 0.0, 0.01, 0.02 ... 3.0] ~ 300

5) 
    
    [0, x, -1] x = [1, 2, 3, 4, 5, 6]

    [1, [0,1...100], [0,1,2,3]] y = X_i_1, x = [0,1,2,3]

    [2, -1, [0.0,0.1...3.0]] x = X_i_2

    [3, [0,1...100], [0.0,0.1...3.0]] 
   
6) Count_1,2,3,4,5,6

In [1]:
import numpy as np
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv("xxx_train.csv")
x_test = pd.read_csv("xxx_test.csv")
x_test = np.array(x_test)
y_train = np.array(df.pop('y'), dtype = 'int')
x_train = np.array(df)

In [3]:
def shift(a, k, uk = 0):
    for i in range(len(a) - k - 1, -1, -1):
        a[i + k] = a[i]
    for i in range(k):
        a[i] = uk
    return a

def feature_engineering(A):
    n, m = A.shape
    y = []
    maps = [-0.19, 1, 2, 3]
    for i in range(n):
        u = []
        sum_marks = [0] * 404
        sum_mixing = [1] * 101
        sup = [0] * 101 * 4
        cnt_marks = [0] * 101 * 4
        AllSumMix = 0
        for j in range(0, m, 3):
            if (A[i][j] == 0):
                shift(sum_marks, 4 * int(A[i][j + 1]))
                shift(cnt_marks, 4 * int(A[i][j + 1]))
                shift(sum_mixing, int(A[i][j + 1]), uk = 1)
            elif (A[i][j] == 1):
                sum_marks[int(A[i][j + 1]) * 4 + int(A[i][j + 2])] += maps[int(A[i][j + 2])]
                cnt_marks[int(A[i][j + 1]) * 4 + int(A[i][j + 2])] += 1
            elif (A[i][j] == 3):
                sum_mixing[int(A[i][j + 1])] += A[i][j + 2]
            else:
                AllSumMix += A[i][j + 2]
                
        ncnt = [0] * 4
        for j in range(101):
            for k in range(4):
                ncnt[k] += cnt_marks[j * 4 + k]
                
        sum_in_cell = [0] * 101
        for j in range(101):
            sum_in_cell[j] = sum(sum_marks[4 * j: 4 * (j + 1)])
            
        feature = []
        for k in range(4):
            s = 0
            for j in range(101):
                s += (sum_marks[j] / sum_mixing[j]) * cnt_marks[j * 4 + k]
            feature.append(s)
        
        u += feature #
        
        u += sum_in_cell
        
        u += ncnt # cnt marks
        
        u.append(sum(cnt_marks)) # sum all marks
        u.append(AllSumMix) # sum mixing in cell
        y.append(u)
    return np.array(y)

In [4]:
alg = Lasso(alpha = 0.007, max_iter = 10000)
resQ = cross_val_score(estimator = alg, X = feature_engineering(x_train), cv = 2, y = y_train, scoring = "neg_mean_absolute_error")
print(-resQ.mean())

1.379361060447689


In [27]:
alg.fit(feature_engineering(x_train), y_train)
RES = alg.predict(feature_engineering(x_test))
file = open("answer", "w")
file.write("id,y\n")
file.write(pd.DataFrame.to_csv(pd.DataFrame(np.round(RES)), header=None))
file = open("answer", "r")
pd.read_csv("answer", index_col = "id")

,y
id,
0,14.0
1,17.0
2,24.0
3,22.0
4,21.0
...,...
49995,13.0
49996,16.0
49997,26.0
